In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
import pandas as pd

In [5]:
links=[]
status=[]

url= 'https://spacenet.tn/74-pc-portable-tunisie?page='
for page in range(1,10):
 response=requests.get(url + str(page))

 if response.status_code==200:
    soup= BeautifulSoup( response.text, 'html.parser')
    
    product_divs = soup.find_all('div', class_='left-product')

#Extract links
    for div in product_divs:
     product_link = div.find('a', href=True)
     if product_link:
        links.append(product_link.get('href'))
        
#Extract status
    divs=soup.find_all('div', class_='product-quantities')
    for div in divs:
        status.append(div.find('label').text)

In [6]:
titre=[]
current_price=[]
old_price=[]
image_product=[]
marque=[]
ref=[]
discount=[]

#Extract Titles
for link in links:
     response = requests.get(link)
     if (response.status_code==200):
      soup = BeautifulSoup(response.text, 'html.parser')
      titre.append(soup.find('h1', class_='h1').text)
#Extract current_price
      current_price.append(soup.find('div', class_='current-price').find('span').text)
#Extract old_price
      i=soup.find('div', class_='product-discount')
      if(i):
       old_price.append(i.find('span').text)
      else:
        old_price.append(" ")
#Extract image_product
      i=soup.find('img', class_='js-qv-product-cover')
      if(i):
        image_product.append(i.get('src'))
      else:
        image_product.append(" ")
#Extract marque
      i=soup.find('img', class_='img img-thumbnail manufacturer-logo')
      if(i):
       marque.append(i.get('alt'))
      else:
        marque.append(" ")        
#Extract ref
      i=soup.find('div', class_='product-reference')
      if(i):
       ref.append(i.find('span').text)
      else:
        ref.append(" ")
#Extract discount
      i=soup.find('span', class_='discount discount-amount')
      if(i):
       discount.append(i.text)
      else:
        discount.append(" ") 

In [16]:
#Store previous data in the dictionary "data"
data = {"Name":titre,"Availability":status,"Ref":ref,"Discount":discount,"Image URL":image_product,"Brand":marque,"Old Price":old_price,"Price":current_price,"product_url":links}
     

In [17]:
#Extract and store info Fiche Technique

columns=["Système d'exploitation", "Mémoire", "Garantie", "Taille de l'écran", "Type de Processeur", "Disque Dur",
"Carte Graphique", "Couleur"]

for link in links:
     response = requests.get(link)
     if (response.status_code==200):
      soup = BeautifulSoup(response.text, 'html.parser')
      dl_tags = soup.find_all('dl')

      for dl in dl_tags:
        dt_elements = [i.text for i in dl.find_all('dt')] 
        dd_elements = [i.text for i in dl.find_all('dd')]
      for i,j in zip(dt_elements, dd_elements):
        if i in columns:
          if i not in data.keys():
            data[i]=[j]
          else:
            data[i].append(j)
        
      

In [18]:
df=pd.DataFrame(data)

In [19]:
df["Store"]="Spacenet"

In [20]:
df["Old Price"]=[int(''.join(filter(str.isdigit, i))) if i != " " else i for i in df["Old Price"]]

In [21]:
df["Price"]=[int(''.join(filter(str.isdigit, i))) for i in df["Price"]]

In [22]:
df["Discount"]=[int(''.join(filter(str.isdigit, i))) if i != " " else i for i in df["Discount"]]

In [23]:
df

,Name,Availability,Ref,Discount,Image URL,Brand,Old Price,Price,product_url,Système d'exploitation,Mémoire,Garantie,Taille de l'écran,Type de Processeur,Disque Dur,Carte Graphique,Couleur,Store
0,Pc Portable Lenovo V15 G4 AMN AMD Athlon 8Go 2...,En stock,82YU0044FE,,https://spacenet.tn/150735-large_default/pc-po...,Lenovo,,729000,https://spacenet.tn/ordinateur-portable/58379-...,Free Dos,8 Go,1 An,15.6 Pouces,AMD,256 Go SSD,AMD Readon,Noir,Spacenet
1,Pc Portable Dell Vostro 3500 i3 11Gén 4Go 1 To...,Rupture de stock,N6501VN3500EMEA01,284000,https://spacenet.tn/67310-large_default/pc-por...,Dell,1123000,839000,https://spacenet.tn/ordinateur-portable/42970-...,Free Dos,4 Go,1 An,15.6 Pouces,Intel Core i3,1 To,Graphique Intégrée,Noir,Spacenet
2,Pc Portable Dell Vostro 3500 i3 11Gén 8Go 1 To...,En stock,N6501VN3500EMEA01-8,,https://spacenet.tn/71540-large_default/pc-por...,Dell,,859000,https://spacenet.tn/ordinateur-portable/43747-...,Free Dos,8 Go,1 An,15.6 Pouces,Intel Core i3,1 To,Graphique Intégrée,Noir,Spacenet
3,Pc Portable Dell Vostro 3500 i3 11Gén 4Go 256G...,En stock,N6501VN3500EMEA01-256SSD,,https://spacenet.tn/112658-large_default/pc-po...,Dell,,859000,https://spacenet.tn/ordinateur-portable/51327-...,Free Dos,4 Go,1 An,15.6 Pouces,Intel Core i3,256 Go SSD,Graphique Intégrée,Noir,Spacenet
4,Pc Portable Dell Vostro 3500 i3 11Gén 4Go 1 To...,En stock,N6501VN3500EMEA01-1T+256SSD,,https://spacenet.tn/112663-large_default/pc-po...,Dell,,879000,https://spacenet.tn/ordinateur-portable/51329-...,Free Dos,4 Go,1 An,15.6 Pouces,Intel Core i3,1 To + 256 Go SSD,Graphique Intégrée,Noir,Spacenet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Pc Portable Lenovo IdeaPad 3 15ALC6 AMD Ryzen ...,En stock,82KU00J6FG-12,,https://spacenet.tn/118564-large_default/pc-po...,Lenovo,,1539000,https://spacenet.tn/pc-portable-tunisie/52388-...,Free Dos,12 Go,1 An,15.6 Pouces,AMD Ryzen 7,512Go SSD,AMD Readon,Bleu,Spacenet
356,Pc Portable Asus X515EP i5 11Gén 8Go 256 Go SS...,Sur commande,X515EP-BQ676W,150000,https://spacenet.tn/127931-large_default/pc-po...,Asus,1699000,1549000,https://spacenet.tn/pc-portable-tunisie/54327-...,Windows 11,8 Go,2 ans,15.6 Pouces,Intel Core i5,256 Go SSD,Nvidia Geforce,Bleu,Spacenet
357,Pc Portable Asus X515EP i5 11Gén 12Go 256 Go S...,Rupture de stock,X515EP-BQ398W-12,,https://spacenet.tn/125661-large_default/pc-po...,Asus,,1549000,https://spacenet.tn/pc-portable-tunisie/53807-...,Windows 11,12 Go,2 ans,15.6 Pouces,Intel Core i5,256 Go SSD,Nvidia Geforce,Silver,Spacenet
358,Pc Portable Lenovo IdeaPad 3 i5 11Gén 8Go 1To ...,En stock,82H800VJFE,180000,https://spacenet.tn/122241-large_default/pc-po...,Lenovo,1729000,1549000,https://spacenet.tn/pc-portable-tunisie/48293-...,Free Dos,8 Go,1 An,15.6 Pouces,Intel Core i5,1 To,Nvidia Geforce,Bleu,Spacenet


In [24]:
df.columns

Index(['Name', 'Availability', 'Ref', 'Discount', 'Image URL', 'Brand',
       'Old Price', 'Price', 'product_url', 'Système d'exploitation',
       'Mémoire', 'Garantie', 'Taille de l'écran', 'Type de Processeur',
       'Disque Dur', 'Carte Graphique', 'Couleur', 'Store'],
      dtype='object')

In [25]:
df.to_csv('spacenet_products_.csv', mode='a', index=False)